# fftboost: Ensemble-of-Experts Boosting (EEB) â€” Visual Demo

This Colab-ready notebook demonstrates the full EEB pipeline:
- Deterministic training with Huber loss + early stopping
- Ensemble of experts (fft_bin + sk_band)
- Explainable descriptors (bin and band ranges)
- Visualizations: signal, spectrogram, selected bins/bands, stage contributions
- Deterministic artifact save/load with stable SHA256


In [ ]:
!pip install git+https://github.com/pinballsurgeon/fftboost.git --quiet
import numpy as np
import matplotlib.pyplot as plt
from fftboost import FFTBoost, BoosterConfig
from fftboost.io import save_model, load_model


## 1) Generate a synthetic signal (AM 50 Hz tone + noise)

In [ ]:
rng = np.random.default_rng(123)
fs = 1000.0
duration_s = 10.0
n = int(fs*duration_s)
t = np.arange(n)/fs
env = 0.7 + 0.3*np.sin(2*np.pi*0.25*t)
x = env*np.sin(2*np.pi*50.0*t) + 0.4*rng.standard_normal(n)
# Target: RMS per window of the clean amplitude-modulated tone
win_s, hop_s = 0.5, 0.25
win_len, hop = int(win_s*fs), int(hop_s*fs)
y_true_signal = env*np.sin(2*np.pi*50.0*t)
n_win = (n - win_len)//hop + 1
windows = np.lib.stride_tricks.as_strided(y_true_signal, shape=(n_win, win_len), strides=(y_true_signal.strides[0]*hop, y_true_signal.strides[0]))
y = np.sqrt(np.mean(windows**2, axis=1)).astype(np.float64)
x = x.astype(np.float64)
fig,ax = plt.subplots(1,1,figsize=(10,3))
ax.plot(t[:5000], x[:5000], lw=0.8); ax.set_title('Signal (first 5s)'); ax.set_xlabel('s'); ax.set_ylabel('amplitude'); plt.show()


## 2) Configure fftboost (EEB) and activate sk_band

In [ ]:
# Log-spaced band edges from 10 Hz to Nyquist (fs/2)
nyq = fs/2.0
edges = np.logspace(np.log10(10.0), np.log10(nyq), 11)
bands = [(float(edges[i]), float(edges[i+1])) for i in range(len(edges)-1)]
config = BoosterConfig(
    n_stages=32, nu=0.5, ridge_alpha=1e-3, early_stopping_rounds=8,
    loss='huber', huber_delta=1.0, k_fft=6, min_sep_bins=3,
    default_band_edges_hz=bands, sk_n_select=2, sk_kurtosis_boost=0.2
)
model = FFTBoost(config)


## 3) Train with contiguous holdout early stopping

In [ ]:
_ = model.fit(x, y, fs=fs, window_s=win_s, hop_s=hop_s, val_size=0.2, val_gap_windows=1)
yhat = model.predict(x, fs=fs, window_s=win_s, hop_s=hop_s)
fig,ax = plt.subplots(1,1,figsize=(10,3))
ax.plot(y, label='y', lw=1.5); ax.plot(yhat, label='yhat', lw=1.25); ax.legend(); ax.set_title('Target vs Prediction'); plt.show()
print('best_iteration:', model._booster.best_iteration_)


## 4) Visualize selected FFT bins and band proposals

In [ ]:
artifact = model._booster.artifact
freqs = artifact.freqs
bins = []
bands_sel = []
for s in artifact.stages:
    for d in s.descriptors:
        if d.get('type')=='fft_bin':
            bins.append(int(d['bin']))
        elif d.get('type')=='sk_band':
            bands_sel.append(tuple(d['band_hz']))

fig, ax = plt.subplots(1,1,figsize=(10,3))
ax.hist([freqs[b] for b in bins], bins=20, color='tab:blue');
ax.set_title('Selected FFT bin frequencies'); ax.set_xlabel('Hz'); plt.show()

if bands_sel:
    lows = [b[0] for b in bands_sel]; highs = [b[1] for b in bands_sel]
    fig, ax = plt.subplots(1,1,figsize=(10,3))
    ax.scatter(lows, highs, s=20, c='tab:orange');
    ax.set_title('Selected band ranges (lo vs hi)'); ax.set_xlabel('lo (Hz)'); ax.set_ylabel('hi (Hz)'); plt.show()


## 5) Artifact determinism (save + load)

In [ ]:
import tempfile, os
tmp = tempfile.mkdtemp()
info = model.save(os.path.join(tmp, 'model'))
artifact2 = load_model(os.path.join(tmp, 'model'))
cfg_dict = artifact2.config or {}
cfg_params = cfg_dict.get('cfg', cfg_dict)
model2 = FFTBoost(BoosterConfig(**cfg_params))
from fftboost.booster import Booster as BoosterLoaded
booster2 = BoosterLoaded(model2.config)
booster2.stages = artifact2.stages
booster2.freqs = artifact2.freqs
booster2.best_iteration_ = len(artifact2.stages) - 1
booster2.__dict__['artifact'] = artifact2
model2._booster = booster2
model2.is_fitted = True
yhat2 = model2.predict(x, fs=fs, window_s=win_s, hop_s=hop_s)
print('SHA256:', info['sha256'])
print('bit-identical predictions:', np.allclose(yhat, yhat2) and (yhat.shape == yhat2.shape))
